In [1]:
# 지도 그리기
import folium
import pandas as pd

In [2]:
# 지도에 마커 표시하기
# 위도 경도의 자료가 있는 데이터를 load
df = pd.read_excel('./dataset/서울지역 대학교 위치.xlsx', index_col = 0)
print(df.head())

# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55, 126.98], zoom_start=12, tiles= 'Stamen Terrain')

                     위도          경도
KAIST 서울캠퍼스   37.592573  127.046737
KC대학교         37.548345  126.854797
가톨릭대학교(성신교정)  37.585922  127.004328
가톨릭대학교(성의교정)  37.499623  127.006065
감리교신학대학교      37.567645  126.961610


In [3]:

for name, lat, lng in zip(df.index, df.위도, df.경도):
    # print(name, lat, lng)
    folium.Marker([lat, lng], popup=name).add_to(seoul_map)
seoul_map.save('./dataset/seoul_c1.html')

In [7]:
# 원형 마커를 추가하는 방법
seoul_map = folium.Map(location=[37.55, 126.98], zoom_start=12, tiles= 'Stamen Terrain')

for name, lat, lng in zip(df.index, df.위도, df.경도):
    # print(name, lat, lng)
    folium.CircleMarker([lat, lng], popup=name,
                       radius = 5, color = 'red', fill = True,
                       fill_color = 'yellow',
                       fill_opacity=0.5
                       ).add_to(seoul_map)
    
seoul_map.save('./dataset/seoul_c2.html')

In [29]:
# 지도 영역에 단계구분도(CHoropleth Map) 표시
import json
df = pd.read_excel('./dataset/경기도인구데이터.xlsx', index_col = '구분')
print(df.head())
df.columns # 컬럼명이 숫자 -> 문자로 변경이 필요
df.columns = df.columns.map(str) # 숫자 -> 문자로 변경
df.columns # 문자(object)로 변경되었는지 확인

          2007    2008    2009    2010    2011    2012    2013    2014  \
구분                                                                       
수원시장안구  287474  285803  290798  293692  290999  291757  300908  301196   
수원시권선구  310553  308177  304097  306783  321176  332633  331773  339835   
수원시팔달구  216945  213235  219833  216503  209772  206794  204805  203479   
수원시영통구  252730  260210  258421  260557  266542  289074  310671  329718   
성남시수정구  256744  248452  242161  241070  236123  233997  228167  223539   

          2015    2016    2017  
구분                              
수원시장안구  299016  296479  293828  
수원시권선구  351053  358393  365653  
수원시팔달구  199180  198515  193311  
수원시영통구  335375  340654  349836  
성남시수정구  219531  232841  236932  


Index(['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017'],
      dtype='object')

In [11]:
# 경기도 시군구 경계 정보를 가져온다.
# 한글 깨짐을 방지하고, encoding UTF-8를 위해서 json lib을 사용
geo_path = './dataset/경기도행정구역경계.json'

try:
    geo_data = json.load(open(geo_path, encoding='utf-8'))
except:
    geo_data = json.load(open(geo_path, encoding='utf-8-sig'))

geo_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': 31380,
    'name': '양평군',
    'name_eng': 'Yangpyeong-gun',
    'base_year': 2013},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.56113535909492, 37.6256560539258],
      [127.57290963929243, 37.61087369350498],
      [127.56366973908277, 37.5842624297699],
      [127.52226606559191, 37.5828628697518],
      [127.50048767007603, 37.569036373178626],
      [127.47687473909174, 37.574448241913856],
      [127.47565866803647, 37.60512112601634],
      [127.44699730711024, 37.64497584741164],
      [127.4272349102434, 37.66284420898682],
      [127.40156700708162, 37.64806970784708],
      [127.3732642199939, 37.6453987554284],
      [127.3542931884407, 37.6250006376975],
      [127.34360057873045, 37.588997440929354],
      [127.31002843450217, 37.53523876142835],
      [127.30923294884336, 37.5135706079458],
      [127.32809611134748, 37.53133849534739],
      [127.3663244453597, 37.5

In [58]:
g_map = folium.Map(location=[37.55, 126.98], zoom_start=12, tiles= 'Stamen Terrain')

year = '2017'

# 경계구분을 추가
cp = folium.Choropleth(geo_data = geo_data, # 지도 경계
                  data =  df[year],     # 2017년 자료
                  columns = [df.index, df[year]],  # 열 지정
                  fill_color = 'YlOrRd', fill_opacity=0.6, line_opacity=0.3,
                  threshold_scale=[10000, 100000, 300000, 500000, 700000],  # 인구별 색 구분을 위해서 정함
                  key_on ='feature.properties.name' # json의 구조를 기입
                      ).add_to(g_map)   
  
# 마우스로 클릭한 곳에 지역 이름 표시하기
folium.LayerControl().add_to(g_map)
cp.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], labels=False)
)

g_map.save('./dataset/g_mail'+year+'.html')

In [59]:
for feature in geo_data['features']:
    print(feature['properties']['name'])

양평군
가평군
연천군
여주시
포천시
양주시
광주시
화성시
김포시
안성시
이천시
파주시
용인시수지구
용인시기흥구
용인시처인구
하남시
의왕시
군포시
시흥시
오산시
남양주시
구리시
과천시
고양시일산서구
고양시일산동구
고양시덕양구
안산시단원구
안산시상록구
동두천시
평택시
광명시
부천시오정구
부천시소사구
부천시원미구
안양시동안구
안양시만안구
의정부시
성남시분당구
성남시중원구
성남시수정구
수원시영통구
수원시팔달구
수원시권선구
수원시장안구
